<a href="https://colab.research.google.com/github/gopimandala/newdevops/blob/master/TL_pizza_steak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
model_url = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2'

In [ ]:
# Get data (10% of labels)
import zipfile

# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2022-03-23 06:54:28--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.191.128, 209.85.147.128, 142.250.136.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.191.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   187MB/s    in 0.9s    

2022-03-23 06:54:29 (187 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
import os 
#os.listdir('10_food_classes_10_percent/train/ramen')


In [ ]:
from PIL import Image
im = Image.open('10_food_classes_10_percent/train/ramen/1433981.jpg')
width, height = im.size
width, height

(512, 512)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
traindir = '10_food_classes_10_percent/train'
testdir = '10_food_classes_10_percent/test'

trainidg = ImageDataGenerator(rescale = 1./255)
testidg = ImageDataGenerator(rescale = 1./255)

# trainidg = ImageDataGenerator()
# testidg = ImageDataGenerator()

traindata = trainidg.flow_from_directory(traindir,target_size=(224,224), batch_size=32, class_mode = 'categorical')
testdata = trainidg.flow_from_directory(testdir,target_size=(224,224), batch_size=32, class_mode = 'categorical')

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
# define feature extraction layer
fe_layer = hub.KerasLayer(model_url, trainable = False, input_shape = (224,224,3))
model = tf.keras.Sequential([
                             fe_layer,
                             layers.Dense(traindata.num_classes, activation = 'softmax')
])
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')
model.fit(traindata, epochs=5, validation_data = testdata)

Epoch 1/5
24/24 [==============================] - 43s 1s/step - loss: 1.7930 - accuracy: 0.4733 - val_loss: 1.1980 - val_accuracy: 0.7340
Epoch 2/5
24/24 [==============================] - 28s 1s/step - loss: 0.9990 - accuracy: 0.7573 - val_loss: 0.8087 - val_accuracy: 0.7964
Epoch 3/5
24/24 [==============================] - 29s 1s/step - loss: 0.7476 - accuracy: 0.8200 - val_loss: 0.6750 - val_accuracy: 0.8176
Epoch 4/5
24/24 [==============================] - 28s 1s/step - loss: 0.6313 - accuracy: 0.8387 - val_loss: 0.6084 - val_accuracy: 0.8268
Epoch 5/5
24/24 [==============================] - ETA: 0s - loss: 0.5508 - accuracy: 0.8667

KeyboardInterrupt: ignored

In [ ]:
traindata.class_indices

In [ ]:
traindata.samples

In [ ]:
preddir = '10_food_classes_10_percent/predict'

predidg = ImageDataGenerator(rescale = 1./255)

preddata = predidg.flow_from_directory(preddir,target_size=(224,224), batch_size=32, class_mode = None)

In [ ]:
ypred = model.predict(preddata)
ypred.tf.argmax()

In [ ]:
ypred = model.predict(preddata)
tf.argmax(ypred,axis=1)

In [ ]:
labels = traindata.class_indices
labels = dict((v,k) for k,v in labels.items())
import numpy as np
predicted_class_indices=np.argmax(ypred,axis=1)
predictions = [labels[k] for k in predicted_class_indices]
predictions